In [1]:
import weaviate
# "http://3.83.67.48:8000/api/health";
client = weaviate.Client(
    url="http://3.83.67.48:8080",
)

d:\Softwares\miniconda\installation\envs\verba\lib\site-packages\weaviate\warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(


In [9]:
schema_list=client.schema.get()

In [10]:
len(schema_list['classes'])

7

### View list of classes

In [11]:
for name in schema_list['classes']:
    print(name['class'])

VERBA_Suggestion
VERBA_MockExamQuestion_text2vec_openai
VERBA_Chunk_text2vec_openai
VERBA_Cache_text2vec_openai
VERBA_Document_text2vec_openai
VERBA_Config
VERBA_Syllabus_Chapters


### Analyse specific class

In [12]:
schema_list['classes'][6]

{'class': 'VERBA_Syllabus_Chapters',
 'description': 'UPSC Entire syllabus in the form of chapters',
 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
  'cleanupIntervalSeconds': 60,
  'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
 'multiTenancyConfig': {'enabled': False},
 'properties': [{'dataType': ['text'],
   'description': 'Number of the chapter in book',
   'indexFilterable': True,
   'indexSearchable': True,
   'name': 'chapter_number',
   'tokenization': 'word'},
  {'dataType': ['text'],
   'description': 'Name of the chapter',
   'indexFilterable': True,
   'indexSearchable': True,
   'name': 'chapter_name',
   'tokenization': 'word'},
  {'dataType': ['text[]'],
   'description': 'List of sub topics in the chapter',
   'indexFilterable': True,
   'indexSearchable': True,
   'name': 'subtopics',
   'tokenization': 'word'},
  {'dataType': ['text'],
   'description': 'pagenumbers from start to the end in book',
   'indexFilterable': True,
   'indexS

### create schema for chapter class

In [38]:
#'clas# Schema for Mock Exam Questions
SCHEMA_SYLLABUS_CHAPTERS = {
    "classes": [
        {
            "class": "VERBA_Syllabus_Chapters",
            "description": "UPSC Entire syllabus in the form of chapters",
            "properties": [
                {
                    "name": "chapter_number",
                    "dataType": ["text"],
                    "description": "Number of the chapter in book",
                },
                {
                    "name": "chapter_name",
                    "dataType": ["text"],
                    "description": "Name of the chapter",
                },
                {
                    "name": "subtopics",
                    "dataType": ["text[]"], 
                    "description": "List of sub topics in the chapter",
                },
                {
                    "name": "pagenumber_start_end",
                    "dataType": ["text"],
                    "description": "pagenumbers from start to the end in book",
                },
                {
                    "name": "ch_id",
                    "dataType": ["text"],
                    "description": "Unique ID of each chapter",
                },
                {
                    "name": "chapter_content",
                    "dataType": ["text"],
                    "description": "content of the entire chapter in markdown format",
                },
            ],
            # ... You can add vectorizer configuration if needed 
        }
    ]
}

In [39]:
client.schema.create(SCHEMA_SYLLABUS_CHAPTERS)

In [30]:
results=client.query.get(
    "MockExamQuestion",
    ["question", "options", "answer_key", "year", "topic", "description", "question_number"],
).with_limit(4).do()

In [ ]:
results

### prepare chapter objects

In [6]:
pwd

'd:\\UPSC-AI\\2024-weaviate\\Verba-1.0.0'

In [26]:
#json_file='d:\UPSC-AI\Data\AI_Mentor\GS-1\json-gemini\India_Ancient_Past_2017.json'
#json_file='d:\\UPSC-AI\\Data\\AI_Mentor\\GS-1\\json-gemini\\India_Ancient_Past_2017.json'
json_file='d:\\UPSC-AI\\Data\\AI_Mentor\\GS-1\\json-gemini\\History_of_Medieval_India_sathish_chandra.json'

In [19]:
data=extract_json_data(json_file)

NameError: name 'extract_json_data' is not defined

In [45]:
import json
import re

def parse_custom_json(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            content = file.read()

        # Extract file_name
        file_name_match = re.search(r'"file_name":\s*"([^"]*)"', content)
        file_name = file_name_match.group(1) if file_name_match else "Unknown"

        # Extract chapters
        chapter_pattern = re.compile(r'\{([^{}]*"chapter_number":[^{}]*"chapter_name":[^{}]*"subtopics":\s*\[[^\]]*\][^{}]*"pagenumber_start_end":[^{}]*"ch_id":[^{}]*"chapter_content":\s*"(.*?)"\s*\})', re.DOTALL)
        chapters = chapter_pattern.findall(content)

        # Process each chapter
        processed_chapters = []
        for chapter in chapters:
            chapter_dict = {}
            chapter_content = chapter[-1]  # Last group is the chapter_content
            chapter_metadata = chapter[0] + chapter_content  # Combine for easier regex

            for field in ["chapter_number", "chapter_name", "subtopics", "pagenumber_start_end", "ch_id", "chapter_content"]:
                if field == "subtopics":
                    subtopics_match = re.search(r'"subtopics":\s*(\[[^\]]*\])', chapter_metadata)
                    if subtopics_match:
                        subtopics_str = subtopics_match.group(1)
                        subtopics = json.loads(subtopics_str)
                        chapter_dict[field] = subtopics
                elif field == "chapter_content":
                    chapter_dict[field] = chapter_content.strip()
                else:
                    match = re.search(f'"{field}":\s*"([^"]*)"', chapter_metadata)
                    if match:
                        chapter_dict[field] = match.group(1)

            processed_chapters.append(chapter_dict)

        # Construct the final JSON structure
        result = {
            "file_name": file_name,
            "metadata": processed_chapters
        }

        return result

    except Exception as e:
        print(f"An error occurred while parsing the file: {str(e)}")
        return None

# Usage
#filename = r"d:\UPSC-AI\Data\AI_Mentor\GS-1\json-gemini\India_Ancient_Past_2017.json"
#filename = r"d:\UPSC-AI\Data\AI_Mentor\GS-1\json-gemini\History_of_Medieval_India_sathish_chandra.json"
filename = r"d:\UPSC-AI\Data\AI_Mentor\GS-1\json-gemini\Modern_India_Bipin_Chandra.json"

result = parse_custom_json(filename)

if result:
    print(json.dumps(result, indent=2))
else:
    print("Failed to parse the file.")

{
  "file_name": "Modern India - Bipin Chandra",
  "metadata": [
    {
      "chapter_name": "The Decline of the Mughal Empire",
      "subtopics": [],
      "pagenumber_start_end": "",
      "ch_id": "H53",
      "chapter_content": "# 1 The Decline Of The Mughal Empire\n\nTHE great Mughal Empire, the envy of its contemporaries, for almost two centuries, declined and disintegrated during the first hair of the 18th century. \n\nThe Mughal Emperors lost their power and glory and their empire shrank to a few square miles around Delhi. In the end, in 1803, Delhi itself was occupied by the British array and the proud Mughal Emperor was reduced to the status of a mere pensioner of a foreign power. A study of the process of decline of this great Empire is most instructive. It reveals some of the defects and weaknesses of India\"s medieval social, economic and political structure which were responsible for the eventual subjugation of the country by the English East India Company.\n\nThe unity 

<>:34: DeprecationWarning: invalid escape sequence '\s'
<>:34: DeprecationWarning: invalid escape sequence '\s'
C:\Users\gkiran\AppData\Local\Temp\ipykernel_19040\4053469680.py:34: DeprecationWarning: invalid escape sequence '\s'
  match = re.search(f'"{field}":\s*"([^"]*)"', chapter_metadata)


In [46]:
#result['metadata'][0]['ch_id']

result['metadata'][0]['chapter_content']

'# 1 The Decline Of The Mughal Empire\n\nTHE great Mughal Empire, the envy of its contemporaries, for almost two centuries, declined and disintegrated during the first hair of the 18th century. \n\nThe Mughal Emperors lost their power and glory and their empire shrank to a few square miles around Delhi. In the end, in 1803, Delhi itself was occupied by the British array and the proud Mughal Emperor was reduced to the status of a mere pensioner of a foreign power. A study of the process of decline of this great Empire is most instructive. It reveals some of the defects and weaknesses of India"s medieval social, economic and political structure which were responsible for the eventual subjugation of the country by the English East India Company.\n\nThe unity and stability of the Empire had been shaken up during the long and strong reign of Aurangzeb; yet in spile of his many harmful policies, the Mughal administration was still quite efficient and the Mughal army quite strong at the time,

In [49]:
result['metadata'][2]['subtopics']

[]

In [50]:
from IPython.display import Markdown

Markdown(result['metadata'][2]['chapter_content'])

# 3 The Beginnings Of European Settlements

I
NDIA'S trade relations with Europe go back to the ancient days of the Greeks. During the Middle Ages trade between Europe and India and SouthEast Asia was carried on along several loutes. One was by sea along the Persian Gulf, and from there overland' dirough Iraq and Turkey, and then again by sea to Venice and Genoa. A second was via the Red Sea and then overland to Alexandria in Egypt and front there by sea to Venice and Genoa. A third, less frequented overland route lay through the passes of the North-West frontier of India, across Central Asia, and Russia to the Baltic. The Asian part of the trade was carried on mostly by Arab merchants and sailors, while the Mediterranean and European part was the virtual monopoly of the Italians. Goods from Asia to Europe passed through many states and many hands. Every state levied tolls and duties while every merchant made a substantial profit, There were many other obstacles, such as pirates and natural calamities on the way. Yet the trade remained highly profitable. This was mostly due to the pressing demand of the people of Europe for Eastern spices which fetched high prices in European markets. The Europeans needed spices because they lived on salted and peppered meat during the winter months, when there was little grass to feed the cattle, and only a liberal use of spices could make this meat palatable. Consequently, European food was as highly spiced as Indian food till the 17th century, The old trading routes between the East and the West came under Turkish control after the Ottoman conquest of Asia Minor and the capture of Constantinople in 1453. Moreover, the merchants of Vcnlce and Genoa monopolised the trade between Europe and Asia and refused to let the new nation states of Western Europe, particularly Spain and Portugal, have any share in the trade through these old routes.

But the trade with India and Indonesia was too highly priced by the West Europeans to be so easily given up. The demand for spices was pressing and the profits to be made in their trade inviting. The reputedly fabulous wealth of India was an additional attraction as there **was an** acute shortage of gold all oyer Europe, and gold **was essential** os a medium of exchange if trade was to grow unhampered. The West European states and merchants therefore began to search for new and safer sea routes to India and the Spice Islands of Indonesia, then known as the East Indies. They wanted to break the Arab and Venetian trade monopolies, to bypass Turkish hostility, and to open direct trade relations with the East. They were well-equipped to do so as great advances in ship-building and the science of navigation had taken place during the 15th century. Moreover, the Renaissance had generated a great spirit of adventure among the people of Western Europe.

Tbe first steps were taken by Portugal and Spain whose seamen, sponsored and controlled by their governments, began a great era of geographical discoveries. In 1494, Columbus of Spain set out to reach India and discovered America instead. In 1498, Vasco da Gama of Portugal discovered a new and all-sea route from Europe to India. He sailed round Africa via the Cape of Good Hope and reached Calicut. He returned with a cargo which sold for 60 times the cost of his voyage. 

These and other navigational discoveries opened a new chapter in the history of the world. Adam Smith wrote later that the discovery of America itnd the Cape route to India were "the two greatest and most important events recorded in the history of mankind." The 17th and 18th centuries were to witness an enormous increase in world trade. The vast new continent of America was opened to Europe and relations between Europe and Asia were completely transformed. The new continent was rich in precious metals. Its gold and silver poured into Europe where they powerfully stimulated trade and provided some of the capital which was soon to make European nations the most advanced in trade, industry and science. Moreover, America was to provide an inexhaustible market for European manufacturers.

Another major source of early capital accumulation or enrichment for European countries was their penetration of Africa in the middle of the 15th century. In the beginning, gold and ivory of Africa had attracted the foreigner. Very soon, however, trade with Africa centred around the slave trade. In the 16th century this trade was a monopoly of Spain and Portugal. Later it was dominated by Dutch, French and British merchants Year after year, particularly after 1650, thousands of Africans were sold as slaves in the West Indies and in North and South America. The slave ships carried manufactured goods from Europe to Africa, exchanged them on the coast of Africa for Negroes, took these slaves across the Atlantic and exchanged them for the colonial produce of plantations or mines, and finally brought back and sold this produoe in Europe. It was on the immense profits of this triangular trade that the commercial supremacy of England and France was to be based.

The demand for slaves on the sugar, cotton and tobacco plantations and mines of the Western hemisphere was inexhaustive as the hard conditions of work and inhuman treatment of the slaves led to high mortality. Moreover, the limited population of Europe could not have supplied the cheap labour needed for the full exploitation of the land and mines of the New World, While no exact record of the number of Africans sold into slavery exists, historians* estimate has ranged between 15 and 50 millions.

While loss of people on a massive scale led to the crippling of African countries and societies, a great deal of West European and North American prosperity was based on the slave trade and the plantations worked by slave labour. Moreover, profits of slave trade and slave-worked plantations provided some of the capital which financed the Industrial Revolution in the 18th and 19th centuries. A similar 

## The Beginnings Op European Settlements 49

role was later played by the wealth extracted from India.

Slavery was later abolished in the 19th century after it had ceased to play an important economic role, but it was openly defended and praised as long as it was profitable. Monarch®, ministers, members of Parlia* ment, dignitaries of the church, leaders of public opinion, and merchants and industrialists supported the slave trade. For example, in Britain, Queen Elizabeth, George III, Edmund Burke, Nelson, Gladstone, Disraeli and Carlyle were some of the defenders and apologists of slavery.

In the 16th century, European merchants and soldiers also began the long process of first penetrating and then subjecting Asian lands to their control. In the process, the prosperity of the Italian towns and merchants waB destroyed as commerce and then political power gradually shifted Westward towards the Atlantic coast.

Portoga.1 had a monopoly of the highly profitable Eastern trade for nearly a century. In India, she established her trading settlements at Cochin, Goa, Dlu, and Daman. From the beginning the Portuguese combined the use of force with trade. In this they were helped by (he superiority of their armed ships which enabled them to dominate the seas. A handful of Portuguese soldiers and sailors could maintain their position on the seas against the much more powerful land powers of India and Asia. Beside^, they also saw that they could take advantage of the mutual rivalries of the Indian princes to strengthen their position. They intervened in the conflict between the ruler; of Calicut and Cochin to establish their trading centres and forts on the Malabar coast From here they attacked and destroyed Arab shipping, brutally killing hundreds of Arab merchants and seameh, By threatening Mughal shipping, they sflso succeeded jn securing many trading Concessions from the Mughal Emp^fors, Under the viceroyalty of Alfanso d" Albuquerque, who captured Goa in |3J0; the Portuguese established their domination over Hie entire Asian coast from Hormuz in the Persian Gulf to Malacca in Malaya and the Spice Islands in Indonesia. They seized Indian territories on the coast and waged constant war to expand their trade and dominions and safeguard their trade monopoly from their European rivals, Nor did they shy away from piracy and plunder. In the words of James Mill, the famous British historian of the 19th century: "The Portuguese followed their merchandise as their chief occupation, but like the English and the Dutch .. of the same period, had no objection to plunder, when it fell in their way." The Portuguese were intolerant and fanatical in religious matters. They indulged in forcible conversion "offering people the alternative of Christianity or sword." Their approach in this respect was particularly hateful to people of India where religious tolerance was the rule. They also indulged in inhuman cruelties and lawlessness. In spite of their barbaric behaviour their possessions in India survived for a century because they enjoyed control over the high seas, their soldiers and administrators maintained strict discipline, and they did not have to face the might of the Mughal Empire as South India was outside Mughal influence. They clashed with the Mughal power in Bengal in 1631 and were driven out of their settlement at Hugli. Their hold over the Arabian sea had 50 **MODERN** INDIA
already been weakened by the English and their influence in Gujarat had become negligible by this time.

Portugal was, however, incapable of maintaining for long its trade monopoly or its dominions in the East. Its population was less than a million, its Court was autocratic and decadent, its merchants enjoyed much less power and prestige than its landed aristocrats, it lagged behind in the development of shipping, a >d it followed a poliey of religious intolerance. The Portuguese and the Spanish had left the English and the Dutch far behind during the 15th century and the first half of the 16th century. But, in the latter half of the 16th century, England and Holland, and later France, all growing commercial and naval powers, waged a fierce struggle against the Spanish and Portuguese monopoly of world trade. In this struggle the latter had to go under. Portugal had become a Spanish dependency in 1580. In 1588 the English defeated the Spanish fleet called the Armada and shattered Spanish naval supremacy for ever. This enabled the English ^nd the Dutch merchants to use the Cape of Good Hope route to India and so to join in the race for empire in the East. In the end, the Dutch gained control over Indonesia and the British over India,, Ceylon, and Malaya.

The Dutch had Toe long been dealing in Eastern produce which they bought in Portugal and sold all over Northern Europe. This had fed them to develop better ships, scientific sailing techniques, and efficiept business methods and organisation. Their revolt against the Spanish domination of their homeland, the Netherlands, and Portugal"s merger with Spain mads them look for alternative sources of spices. In 1595, four Dutch ships sailed to India via the Cape of Good Hope. In 1602, the Dutch East India Company was formed and the Dutch States General —the Dutch parliamentgave it a Charter empowering it to make war, conclude treaties, acquire territories and build fortresses.

The main interest of the Dutch lay not in India but in the Indonesian Islands of Java, Sumatra, and the Spice Islands where spices were produced. They soon turned out the Portuguese from the Malay Straits and the Indonesian Islands and, in 1623, defeated English attempts to establish themselves there. It appeared at the time that the Dutch had successfully seized the most important profitable part of Asian trade. They did not, however, entirely abandon Indian trade. They also established trading depots at Surat, Broach, Cambay, and Ahmedabad in Gujarat in West India, Cochin in Kerala, Nagapatam in Madras, Masulipatam in Andhra, Chinsura in Bengal, Patna in Bihar, and Agra in Uttar Pradesh. In 1658 they also conquered Ceylon from the Portuguese. They exported indigo, raw silk, cotton textiles, saltpetre, and opium from India. Like the Portuguese they treated the people of India cruelly and exploited them ruthlessly.

The English merchants too looked greedily on the Asian trade. The success of the Portuguese, the rich cargoes of spices, calicoes, silk, gold, pearls, drugs, porcelain,and ebony they carried, and the high profits they made inflamed the imagination of the merchants of England and made them impatient to participate in such profitable commerce. But, till the end of the 16th century, they WC TO too weak to challenge the naval might of Portugal and Spain. For over 50 years they 

## The Beginnings Op European Settlements 51

searched without success for an alternative passage to India. Meanwhile they gathered strength on the sea. In 1579, Drake sailed around the world. In 1588, the defeat of the Spanish Armada led to the opening of the sea-passage to the East.

An English association or company to trade with the East .was formed in 1599 under the auspices of a group of merchants known as the Merchant Adventurers. The company was granted a Royal Charter and the exclusive privilege to trade in the East by Queen Elizabeth on 31 December 1600 and was popularly known as the East India Company. From the beginning, it was linked with the monarchy: 
Queen Elizabeth (1558-1603) was one of the shareholders of the company.

The first voyage of the Bnglish East India Company was made in 1601 when its ships sailed to the Spice Islands of Indonesia. In 1608 it decided to open a factory,.the name given at the time to a trading depot, at Surat on the West coast of India and sent Captain Hawkins to Jahangir"s Court to obtain Royal favours. Initially, Hawkins was received in a friendly manner. He was given a **man** sab of 400 and a jagir. Later, he was expelled from Agra as a result of Portuguese intrigue. This .convinced the English of the need to overcome Portuguese influence at the Mughal Court if they were to obtain any concessions from the Imperial Government. They defeated a Portuguese naval squadron at Swally near Surat in 1612 and then again in 3 614. These victories led the Mughals to hope that in view of their naval weakness they could use the English to counter the Portuguese on the sea. Moreover, the Indian merchants would certainly benefit by competition among their foreign buyers. Consequently, the English Company was given permission by a Royal *farman* to open factories at several places on (he West coast.

The English were not satisfied with this concession. In 1615 their ambassador Sir Thomas Roe reached the Mughal Court. They also exerted pressure on the Mughal authorities by taking advantage of India"s naval weakness and harassing Indian traders and shipping to the Red Sea and to Mecca. Thus, combining entreaties with threats, Roe succeeded in getting an Imperial *farman* to trade and establish factories in all parts of the Mughal Empire. Roe's success further angered the Portuguese and a fierce naval battle between the two countries began in 1620. It ended in English victory. Hostilities between the two came to an end in 1630. In 1662 the Portuguese gave the Island of Bombay to King Charles II of England as dowry-for marrying a Portuguese Princess. Eventually, the Portuguese lost alt their possessions in India except Goa, Din and Daman. The Dutch, the English, and the Marathas beflefitted, the Marathas capturing Salsette and Bassein in 1739.

The English Company fell out with the Dutch Company over division of the spice trade of the Indonesian Islands. Ultimately, the Dutch nearly expelled the English from the trade of the Spice Islands and the latter were compelled to concentrate on India where the situation was more favourable to them- The intermittent war in India between the1two powers, which had begun in 1654, ended in 1667, when the English gave up all claims to Indonesia while the Dutch agreed to leave alone the English settlements in 52 **MODERN** INDIA
India. The English, however, continued. their efforts to drive out the Dutch from the Indian trade and by 1795 'hey had expelled the Dutch from their last possession in India.

## The Groi»<H Of {He East India Company'S Trade And Influence, 1W0-1744

![59_Image_0.Png](59_Image_0.Png)

The English East Company had vety humble beginnings in India. Surat was the centre of its trade till 1687. Throughout this period the English remained petitioners before the Mughal authorities. By 1623 the; had established factories at Surat, Broach, Ahmedabad, Agra, and Masulipatam. From the very beginning, the English trading company tried to combine trade and diplomacy with war and control of the territory where their factories were situated. In fact, already in 161V Roe had given to the English authorities tbe advice that was to Jay the pattern *tot**
future British relations with India. "Assure you", he wrote, "I know these people are best treated with the sword in one hand and the Caducean (a rod carried by a messenger) in the other." He added that the English should depend "upon the same ground that we began and by which we subsist, feaf."

A View of the City of Surat In the Early Seventeenth Century Courtesy: National Archives of India, New Dtthl
In 1625 the Company"s authorities at Surat made an attempt to fortify their factory but the chiefs of the English factory were immediately imprisoned and put in irons by the local authorities of the Mughal Empire which was still in its vigour. Similarly, when the Company"s English rivals made piratical attacks on Mughal shipping, the Mughal authorities imprisoned in retaliation the President of the Company at Surat and members of his Council and released them only on payment of £ 18,000.

Conditions in the South were more favourable to the English as they did not 

## The Beginnings Op European Settlements 53

have to face a strong Indian Government there. The great Vijaya- n^gar Kingdom had been overthrown in 1565 and its place taken by a number of petty and weak states, It was easy to appeal to their greed or overawe them with armed strength. The English opened their first factory in the South at Masulipatam in 1611. But they soon shifted the centre of their activity to Madras the lease of which was granted to thc;zn by the local Raja in 1639. Madras was then a strip of coastal territgjry six miles long and one mile broad. The Raja authorised them td fortify 
(he place, to administer it, and to coin money on condition of payment to him of half of the customs revenue of the port. Here the English built a small fort around 

![60_image_0.png](60_image_0.png)

By the end of"the 17th ' **Century** the English Company was claiming ftill sovereignty over Madras and was ready to fight in, defence of the claim. 

Interestingly enough, from the very beginning this Company of profit- seeking merchants was also determined to make Indians pay for the conquest of their own country. For example, the Court of Directors of the Company wrote to the Madras authorities in 1683:
would have you to .strengthen, and fortify our Fort and Town (Madras) by degrees, that it may be terrible against the as&ault of any Indian Prince and tbe Dutch power of India .... **But we mutt needs desire you so to continue your bosincaa (but with** 
all gentleness) that tbe inhabitants may pay the full charge of all repairs and fortifications 
.......................................................................................................

The Island of Bombay was acquired by the East India'Company from Portugal in 1668 and was immediately fortified. In Bombay the English found a large and easily defended port For thatrea&on, and because English trade was threatened at the time by the rising Maratha power, Bombay soon superceded Surat as the headquarters of the Company on the West Coast.

In Eastern' India, the English Company had opened its first factories hi Orissa in 1633. In 1651 it was given permission to trade at Hugti in Bengal. It soon opened factories at Patna. Balasore, Paoca and other places in Bengal and Bihar. 

54 **MODERN** INDIA
It now desired that in Bengal too it should have an independent settlement. 

Moreover, their easy success in trade and in establishing independent and fortified settlements at Madras and at Bombay, and the preoccupation of Aurangzeb with the anti-Maratha campaigns led the English to abandon the role of humble petitioners. They now dreamt of establishing political power in India which woi>ld enable them to compel the Mughals to allow them a free hand in trade, to force Indians to sell cheap and buy dear, to keep the rival European traders out, and to make their trade independent of the policies of the Indian powers. Political power would also make it possible for them to appropriate Indian revenues and thus to conquer the country wiLh its own resources, Such plans were explicitly put forward at the time. The Governor of Bombay, Gerald Aungier, wrote to the Directors of the Company in London, "the time now lequfres you to manage your gena&l commerce with the sword in your hands," In 1687, the Directors advi&ed the Governor of Madras to:
establish such a policy of civil and military power and create and secure such a luge revenue to maintain both as may be the foundation of a large, well- grounded, secure English dominion in India for all lime to Come.

In 1689 they declared; The increase of our revenue is the subject of our care, as much as our trade: "tis that mi'st maintain our force, when twenty accidents may interrupt our trade; "tis that miut make us a nation in India.............................................................

Hostilities between the English and the Mughal Emperor broke out in 1686 after the former had sacked Hugli and declared war on the Emperor. But the English had seriously miscalculated the situation and underestimated Mughal strength. The Mughal Empire under Aurangzeb was even now more than a match for the petty forces of the East India Company. The war ended disastrously for them. They were driven out of their factories in Bengal and compelled to seek refuge in a fever-strickcn island at the mouth of the Ganga. Their factories at Surat, Masulipatam, and Vizagapataoi were seized and their fort at Bombay besciged. Having discovered that they were not yet strong enough to fight the Mughal power, the English once again became humble petitioners and submitted "that the ill crimes they have done may be pardoned." They expressed their willingness to trade under the protection of the Indian rulers. Obviously, they had learnt their lesson. Once again they relied on flattery and humble entreaties to get trading concessions from the Mughal Emperor.

The Mughal authorities readily pardoned the English folly as they had no means of knowing that these hannless-looking foreign traders would one day pose a serious threat to the country, instead they recognised that foreign trade carried on by the Company benefited Indian artisans and merchants and thereby enriched the State' treasury. Moreover, the

## 55 Modern Thdta

English, though weak on land, were, because of their navat supremacy, capable of completely ruining Indian trade and shipping to Iran, West Asia, Northern and Eastern Africa and East Asia. Aurangzeb therefore permitted them to resume trade on payment of Rs. 150,000 as compensation. In 1691 the Company was granted exemption from the payment of custom duties in Bengal in return for Rs. 3,000 a year. In 1698, the Company acquired the zamindari of the three villages Sutanati, Kalikata, and Govindpur where it built Fort William around its factory. 

The villages Boon grew into a city which came to be known as Calcutta. In 1717 the Company secured from Emperor Farrukh Siyar a *farman* confirming .the privileges granted in 1691 and extending them to Gujarat and the Deccan. But during the first half of the 18th century Bengal was ruled by strong Nawabs such as Murshid Quli Khan and Alivardi Khan. They exercised strict control over the English traders and prevented them from misusing their privileges. Nor did they allow them to strengthen fortifications at Calcutta or to rule the city independently. Here the East India Company remained a mere zamindar Of the Nawab, Even though the political ambitions of the Company were frustrated, its commercial affairs flourished as never before. Its imports from India into England increased from £ 500,000 in 1708 to £ 1,795,000 in 1740. This increase was recorded in spite of the fact that the English Government forbad^ the use of Indian cotton and silk textiles iii England in order to protect the English textile industry and to prevent export of silver from England to Tndia. Thus at a Jime when the English were pleading for free trade in India they were restricting freedom of trade in their own country and denying access to Indian manufactures.

British settlements in Madras, Bombay, and Calcutta became the nuclei of flourishing cities. Large numbers of Indian merchants and bankers were attracted to these cities. This was due partly to the new commercial opportunities available in these cities and partly to the unsettled conditions and insecurity outside them, caused by the break-up of the Mughal Empire. By the middle of the 18 th century, the population of Madras had increased to 300,000, of Calcutta to 200,000 and of Bombay to 70,000. It should also be noted that these three cities contained fortified English settlements; they also had immediate access to the sea where English naval power remained far superior to that of the Indians. In case of conflict with any Indian authority, the English could always escape from these cities to the sea. And when a suitable opportunity arose for them to take advantage of the political disorders m the country, they could use these strategic cities as spring-boards for the conquest of India.

The Internal Organisation of the Company The Charter of 1600 granted the East India Company the exclusive privilege of trading East of the Cape of Good Hope for a period of 15 years. The Charter provided for the management of the Company by a committee consisting of a Governor, a Deputy-Governor, and 24 members to be elected by a general body of the merchants forming the Company. This committee later on came to be known as the "Court of Directors" and its members as "Directors".

The East Indian Company soon became the most important trading company of England. Between 1601 and 1612 its rate of profit came to nearly 20 per cent per 56 **MODERN** INDIA
annum. Its profits were derived both from trade and from piracy, there being no clear dividing line between the two at the time. In 1612 the Company made a profit of £ 1,000,000 on a capital of £ 200,000. During the entire 17th century the rate of profit was very high.

But the Company was a strictly closed corporation or a monopoly. No nonmember was allowed to trade with the East or to share in its high profits. 

However, from the very beginning English manufacturers and those merchants who could not secure a place in the ranks of the monopoly companies carried on a vigorous campaign against royal monopolies like the East India Company. But the monarchs threw their influence behind the big companies who gave heavy bribes to them and to other influential political leaders. From 1609 to 1676, the Company gave loans amounting to £ 170,000 to Charles II, In return, Charles II 
granted it a series of Charters confirming its previous privileges, empowering it to build forts, raise troops, make war and peace with the powers of the East, and authorising its servants in India to administer justice to ail Englishmen and others living in English settlements. Thus the Company acquired extensive military and judicial powers.

Many English merchants continued to trade in Asia in spite of the monopoly of the East India Company. They called themselves "Free Merchants" while the Company called them "Interlopers." These Interlopers in the end compelled the Company to take them into partnership, A change of fortunes occurred in 1688 when Parliament became supreme in England as a result of the Revolution of 1688 which overthrew the Stuart king James II and invited William III and his wife Mary to be the joint sovereign of Britain. The "Free Merchants" now began to press their case on the public and the Parliament. The Company defeuded itself by giving heavy bribes to the King, his ministers, and members of the Parliament. 

In one year alone it spent £ 80,000 on bribes, giving the King £ 10,000. In the end, they secured a new Charter in 1693.

But time was running against the Company; its success was shortlived. In 1694, the House of Commons passed a Resolution that "all subjects of England have equal rights to trade in the East Indies, unless prohibited by Act of Parliament." The rivals of the Company founded another Company known as the New Company. It gave a loan of £ 2,000,000 to the Government at a time when the Old Company could offer only £ 700,000. Consequently, the Parliament granted the monopoly of trade with the East to the New Company. The Old Company refused to give up its profitable trade so easily. It bought large shares in the New Company to be able to influence its policies. At the same time its servants in India refused to let the servants of the New Company carry on trade there. Both companies faced ruin as a result of their mutual conflict. Finally, in 1702, the two decided to join forces and together formed a united company. The new company entitled "The Limited Company of Merchants of England trading to the East Indies" came into existence in 1708.

The Government and Organisation of the Company"s Factories in India As the East India Company gradually grew in power and tended to acquire the 

## The Beginnings Op European Settlements 57

status of a sovereign state in India, the organisation of its factories in India too changed and developed accordingly. A factory of the Company was generally a fortified area within which the warehouses (stores), offices, and houses of the Company"s employees were situated. It is to be noted that no manufacture was carried on in this factory.

The Company"s servants were divided into three ranks: writers, factors, and merchants. They all lived and dined together as if in a hostel and at Company"s cost. A writer was paid 10 pounds (100 rupees) a year, a factor 20 to 40 pounds (200 to 400 rupees), and a merchant 40 pounds (400 rupees) or a little more. 

Thus, they were paid Very low salaries. Their real income, for which they were so keen to take service in India, came from the permission the Company granted them to carry on private trade withirt the country while the trade between India and Europe was reserved for the Company.

The Factory with its trade was administered by a Governor-in-Council. The Governor was merely the President of the Council and had no power apart from the Council which took decisions by a majority vote. The Council consisted of senior merchants of the Company.

## The Anglo-French Straggle In South India

The English East India Company"s 'schemes of territorial conquests and political domination, which had been frustrated by Aurangzeb at the end of the 17th Cfntury,1 were revived during the 1740"s because of the visible decline of the Mughal power. Nadir Shah"s invasion had revealed the decay of t^e central authority. But there was not much scope for foreign penetration in Western India where the vigorous Marathas held sway and jn Eastern India where Allvaicii Khan maintained strict control, [n Southern India, however, conditions were gradually becoming favoui- able to foreign adventurers. While central anthoi ity had disappeared j\oin there after Aurangzeb"s death, the strong hand of N i2am-ul-Mulk Asaf Jah was also withdrawn by nis death in 1748. Moreover, the Maratha chiefs regularly invaded Hyderabad and the rest of the South collecting *chaulh.* These raids resulted in politically unsettled conditions and administrative disorganisation. The Carnatic was embroiled m fratricidal wars of succession.

These conditions gave the foreigners an opportunity to expand their political influence and control over the affairs of the SouLh Indian states. But the Fnghsh were not alone in putting forward commercial and political claims. While they had, by the end of the 17th century, eliminated their Portuguese and Dutch rivals, France had appeared as a new rival. For nearly 20 years from 1744 to 1763 the French and the English were to wage a httter war for control over the trade, wealth, and territory of India.

The French East India Company was founded in 1664. It made rapid progress after it was reorganised in the 1720's and soon began to catch up with the English Company. It was firmly established at Chander- nagore near Calcutta and Pondicherry on the East Coast. The latter was fully forlilicd. The French Company had some other factories at several ports on the Ease and the West coasts. It had also acquired control over the islands of Mauritius and Reunion in the Indian Ocean.

58 **MODERN** INDIA
The French East India Company was heavily dependent on the French Government which helped it by giving it treasury grants, subsidies, and bans, and in various other ways. Consequently, it was largely controlled by the Government which appointed its directors after 1723. Moreover, big shares in the Company were held by the nobles and other rentiers who weie more interested in quick dividends than in making the Company a lasting commercial success. So long as the loans and subsidies from the Government, enabled the directors to declare dividends, they did not care much about the success or soundness of its commercial ventures. State control of the Company proved harmful to it in another way. The French state of the time was autocratic, semifeudal, and unpopular and suffeicd from corruption, inefficiency, and instability. 

Instead of being forward-looking it was decadent, bound by tradition, and In general unsuited to the times. Control by such a state could not but be injurious to the interests of the Company.

Tn 1742, war broke out in Europe between France and England. One of the major causes of the war was rivalry over colonies in America. Another was their trade rivalry in India. This rivalry was intensified by the knowledge that the Mughal Empire was disintegrating and so the prize of trade or territory was likely to be much bigger than in the past. Anglo-French conflict in India lasted-for nearly 20 years and led to the establishment of British power in India. The English Company was the wealthier of the two because of its superiority in trade. Tt also possessed naval superiority. Moreover, its possessions in India had been held longer and were better fortified and more prosperous. Materially, therefore, the advantage lay with the British.

The war in Europe between England and France soon spread to India where the two East India Companies clashed with each other. In 1745, the English navy captured French ships off the South-east coast of India and threatened Pondicherry. Dupleix, the French Governor-General at Pondicherry at this time, was a statesman of genius and imagination. Under his brilliant leadership, the French retaliated and occupied Madras in 1746. This led to a very important event of the war. The British appealed to the Nawab of Carnatic, in whose territory Madras was situated, to save their settlement from the French. The Nawab agreed to intervene as he wanted to convince the foreign merchants that he was still the master of his territories, He sent an army against the French to stop the two foreign trading companies from fighting on his soil. And so the 10,000 strong army of the Nawab clashed with a small French force, consisting of 230 Europeans and 700 Indian soldiers trained along Western lines, at St. 

Thome on the banks of the Adyar river. The Nawab was decisively defeated. 

This battle revealed the immense superiority of Western armies over Indian armies because of their belter equipment and organisation. The Indian pike was no match for the Western musket and bayonet, nor the Indian cavalry for the Western artillery. The large but ill disciplined and unwieldly Indian armies could not stand up against the smaller but better disciplined Western armies.

In 1748, the general war between England and France ended and, as a part of the peace settlement, Madras was restored to the English. Though war had ended, the rivalry in trade and over the possessions in India continued and had to be 

## The Beginnings Op European Settlements 59

decided one way or the other. Moreover, the war had revealed to the full the weakness of Indian government and armies and thereby fully aroused the cupidity of both the Companies for territorial expansion in India.

Dupleix now decided to use the lessons he had learnt in the recent war with the Nawab of Carnatic. He evolved the strategy of using the well- disciplined, modern French army to intervene in the mutual quarrels of the Indian princes and, by supporting one against the other, securing monetary, commercial, or territorial favours from the victor. Thus, he planned to use the resources and armies of the local rajas, nawabs, and chiefs to serve the interests of the French Company and to expel the English from India. The only barrier to the success of this strategy could have been the refusal of Indian ruters to permit such foreign intervention. But the Indian rulers were guided not by patriotism, but by narrow-minded pursuit of personal ambition and gain. They had little hesitation in inviting the foreigners to help them settle accounts with their internal rivals.

In 174S, a situation arose in the Carnatic and Hyderabad which gave full scope to Dupleix"s talents for intrigue. In the Carnatic, Chanda Snhib began to conspire against the Nawab, Anwaruddin, while in Hyderabad the death of Asaf Jab, Nizam-uI-Mulk, was followed by civil war between his son Nasir Jang and his grandson Muzaffar Jang. Dupleix seized this opportunity and concluded a secret treaty with Chanda Sahib and Muzaffar Jang to help them with his well-trained French and Indian forces. In 1749, the three allies defeated and killed Anwaruddin in a battle at Ambur. The latter"s son, Muhammad Ali, fled to Tnchinopoly. The rest of the Cainatic passed under the dominion of Chanda Sahib who rewarded the French with a grant of 80 villages around Pondicherry.

In Hyderabad too, the French were successful. Nasir Jang was killed and Muzaffar Jang became the Nizam or Viceroy of the Dcccan. The new Nizam rewarded the French Company by giving it territories near Pondicherry as well as the famous town of Masuhpatam. He gave a sum of Rs. -500,000 to the Company and another Rs. 500,000 to its troops. Dupleix received Rs. 2,000,000 and a jagir worth Rs. 100,000 a year. Moreover, he was made honorary Governor of Mughal dominions on the East coast from the river Krishna to Kanya Kumari. Dupleix stationed his best officer, Bussy, at Hyderabad with a French army. "While the ostensible purpose of this arrangement was to protect the Nizam from enemies, it was really aimed at maintaining French influence at his court. While Muzaffar Jang was marching towards his capital, he was accidentally killed. Bussy immediately raised Salabat Jang, the third son of Nizam-ul-Mulk, to the throne. Tn return, the new Nizam granted the French the area in Andhra known as the Northern Sarkars, consisting of '■he four districts of Mustafanagar, Ellore, Rajahmundry, and Chicacole.

The French power in South India was now at its height, Dupleix"s plans had succeeded beyond his dreams. The French had started out by trying to win Indian states as friends; they had ended by making them clients or satellites.

But the English had not been silent spectators of their rival"s suocesses, To offset French influence and to increase their own, they had been intriguing with 60 **MODERN** INDIA
Nasir Jang and Muhammad Ali. In 1750, they decided to throw their entire strength behind Muhammad Ali. Robert Clive, a young clerk in the Company"s service, proposed that French pressure on Muhammad Ali, besieged at Tnchinopoly, could be released by attacking Arcot, the capital of Carnatic. The proposal was accepted and Clive assaulted and occupied Arcot with only 200 English and 300 Indian soldiers. As expected, Chanda Sahib and the French were compelled to raise the seige of Trichinopoly, The French foices were repeatedly defeated. Chanda Sahib was soon captured and killed. The French fortunes were now at an ebb as their army and its generals had proved unequal to their English counterparts.

Dupleix made strenuous attempts to reverse the tide of French misfortunes. But he was given little' support by the French Government or eveh by the higher authorities of the French East India' Company. Moreover, the high French officials and military and naval commanders constantly quarrelled with one another and with Dupleix. In the end, the French Government, weary of the heavy expense of the war in India and fearing the loss of its American colonies, initiated peace negotiations and agreed in 1754 to the English demand for the recall of Dupleix from India. This was to piove a big blow to the fortunes of the French Company in India, The temporary peace between the two Companies ended in 1756 when another war between England and Fiance broke out. In the very beginning of the war, the English managed to gain control over Bengal. This has been discussed in the next chapter. 

After this event, there was little hope for the French cause in India. The rich resources of Bengal turned the scales decisively in favour of the English. Even though the French Government made a determined attempt this time to oust the English from India and sent a strong force headed by Count de Lally, it was all in vain. The French fleet was driven off Indian waters and the French forces in the Carnatic were defeated Moreover, the English replaced the French as the Nizam"s protectors and secured from him P/lasuIipatam and the Northern Sarkars. The decisive battle of the wai was fought at Wandiwash on 22 January 1760' when the English General Eyre Coot defeated Lally. Within a year the French had lost all their possessions in India The war ended in 1763 with the signing of ihe Treaty of Paris. The French factories m India were restored but they could no longei be fortified or even adequately garrisoned with troops. They could seive only .is centres, of trade; and now the French lived in India under British protection Their dream of Empne in Tndia was at an end. The English, on the other hand, ruled the Indian sea. Freed of all European livals. they could now set about the task of conquering India.

During their struggle with the French and their Indian allies; the English learnt a few important, and valuable lessons. Firstly, that in the' absence of nationalism in the country, they could advance their political schemes by taking advantage of the mutual quarrels of the Indian ruleis. Secondly, the Western trained infantry, European or Indian, armed with modern weapons and backed by artillery could defeat the oldstyle Indian armies with ease in pitched battles, Thirdly, it was proved that the Indian soldier trained ani armed in the European manner made as good a soldier 33 the European And since the Indian soldier too lacked a feeling of nationalism, he could he hired and employed by anyone who was willing to pay him well. The English now set out to create a powerful army consisting of Indian soldiers, called 

## The Beginnings Op European Settlements 61

sepoys, and officered by Englishmen, With this army as its chief instrument and the vast rcsourccs of Indian trade and territories under its command, the English East India Company embarked on an era of wars and territorial expansion.

## E X E R C I S E S

1. Discuss the development of European trade with India from the 15th to 18th centuries. \
2. Trace the growth of trade of the English East India Company and its influence on India from 1500 to 1744, 3. What were the factors which contributed to the Anglo-French struggle in South India? How did it lead to the subversion of Indian political power?

4. Write short notes on :
(a) The Portuguese in India, (b) Trade in spices, (c) The Dutch in India, 
(d) Aurangzeb and the East India Company, (e) The organisation of the English East India Company"s factories in India, (f) Dupleix, (g) The French East India Company.

![69_image_0.png](69_image_0.png)

 Based upon Survey of India map with the permission of the Surveyor General of India, The territorial waters of India extend into the sea to a distance of twelve nautical miles measured from the appropriate base line.

In [51]:
result['metadata']

[{'chapter_name': 'The Decline of the Mughal Empire',
  'subtopics': [],
  'pagenumber_start_end': '',
  'ch_id': 'H53',
  'chapter_content': '# 1 The Decline Of The Mughal Empire\n\nTHE great Mughal Empire, the envy of its contemporaries, for almost two centuries, declined and disintegrated during the first hair of the 18th century. \n\nThe Mughal Emperors lost their power and glory and their empire shrank to a few square miles around Delhi. In the end, in 1803, Delhi itself was occupied by the British array and the proud Mughal Emperor was reduced to the status of a mere pensioner of a foreign power. A study of the process of decline of this great Empire is most instructive. It reveals some of the defects and weaknesses of India"s medieval social, economic and political structure which were responsible for the eventual subjugation of the country by the English East India Company.\n\nThe unity and stability of the Empire had been shaken up during the long and strong reign of Aurangzeb

# Update the UPSC BOOK JSON files 

In [56]:
for i, json_node in enumerate(result['metadata']):
    #print(json_node['subtopics'])
    print(json_node['chapter_number'])

KeyError: 'chapter_number'

In [57]:
for i, json_node in enumerate(result['metadata']):
    properties = {
        "chapter_name": json_node['chapter_name'],
        "chapter_number": 'NA',#json_node['chapter_number'],
        "subtopics": json_node['subtopics'],
        "pagenumber_start_end": json_node['pagenumber_start_end'] ,
        "ch_id": json_node['ch_id'],
        "chapter_content": json_node['chapter_content'],
    }

In [58]:
# Configure a batch process
with client.batch as batch:
    batch.batch_size=5
    for i, json_node in enumerate(result['metadata']):
        properties = {
            "chapter_name": json_node['chapter_name'],
            "chapter_number": 'NA',#json_node['chapter_number'],
            "subtopics": json_node['subtopics'],
            "pagenumber_start_end": json_node['pagenumber_start_end'] ,
            "ch_id": json_node['ch_id'],
            "chapter_content": json_node['chapter_content'],
        }

        client.batch.add_data_object(properties, "VERBA_Syllabus_Chapters")

## Use filter to retrieve specific node or chapter

In [38]:
where_filter = {
    "path": ["ch_id"],
    "operator": "Equal",
    "valueText": "H51",
}

In [39]:
response=client.query.get("VERBA_Syllabus_Chapters", ["ch_id","chapter_content"]).with_where(where_filter).do()
print(json.dumps(response, indent=4))

{
    "data": {
        "Get": {
            "VERBA_Syllabus_Chapters": [
                {
                    "ch_id": "H51",
                    "chapter_content": "# Climax And Disintegration Of The Mughal Empire-\n\nPROBLEMS OF SUCCESSION\nThe last years of Shah Jahan's reign were clouded by a bitter war of succession among his sons. There was no clear tradition of succession\n-among the Muslims or the Timurids. The right of nomination by the ruler had been accepted by some of the Muslim political thinkers.\n\nBut it could not be asserted in India during the Sultanat period. The Timurid tradition of partitioning had not been successful either and was never applied in India.\n\nHindu traditions were also not very clear in the matter of succession. According to Tulsidas, a contemporary of Akbar, a ruler had the right of giving the tika to any one of his sons. But there were many cases among the Rajputs where such a nomination had not been accepted by the other brothers. Thus, Sanga 

#### Use filter to select and get for all required content to extract

In [40]:
response=client.query.get("VERBA_Syllabus_Chapters", ["ch_id" ,]).with_where(where_filter).do()
print(json.dumps(response, indent=4))

{
    "data": {
        "Get": {
            "VERBA_Syllabus_Chapters": [
                {
                    "ch_id": "H51"
                }
            ]
        }
    }
}


#### Aggregate all chapters count

In [59]:
#Aggregate all chapters count
result_agg = (
    client.query
    .aggregate("VERBA_Syllabus_Chapters")
    .with_fields("meta {count}")
    .do()
)
result_agg

{'data': {'Aggregate': {'VERBA_Syllabus_Chapters': [{'meta': {'count': 67}}]}}}

In [41]:
#Aggregate filter data aggregate chapters count
where_filter = {
    "path": ["ch_id"],
    "operator": "Equal",
    "valueText": "H100",
}

result_agg = (
    client.query
    .aggregate("VERBA_Syllabus_Chapters")
    .with_fields("meta {count}")
    .with_where(where_filter)
    .do()
)

print(json.dumps(result_agg, indent=4))

{
    "data": {
        "Aggregate": {
            "VERBA_Syllabus_Chapters": [
                {
                    "meta": {
                        "count": 0
                    }
                }
            ]
        }
    }
}


In [42]:
obj=client.data_object.get(class_name="VERBA_Syllabus_Chapters",limit=2)['objects']

In [43]:
obj

[{'class': 'VERBA_Syllabus_Chapters',
  'creationTimeUnix': 1726616475167,
  'id': '06a21ca4-bae4-48f4-a413-141cc5bcc290',
  'lastUpdateTimeUnix': 1726616475167,
  'properties': {'ch_id': 'H20',
   'chapter_content': "# 20 Central Asian Contact And Mutual Impact\n\n## I Political Aspects\n\nThe period that began in about 200 BC did not witness a large empire like that of the Mauryas, but it is notable for intimate and widespread contacts between Central Asia and India. In the eastern and central parts of India and in the Deccan, the Mauryas were succeeded by several native rulers such as the Shungas, the Kanvas, and the Satavahanas. In north-western India they were succeeded by a number of ruling dynasties from Central Asia. Of them, the Kushans became the most famous.\n\n## The Indo-Greeks\n\nA series of invasions began in about 200 BC. The first to cross the Hindu Kush were the Greeks, who ruled Bactria, or Bahlika, situated south of the Oxus river in the area covered by north Afghan

# Delete class

In [ ]:
#client.schema.delete_class('VERBA_Document_text2vec_cohere')

In [7]:
client.schema.delete_class('Spacefarer')